# Data Preprocessing Pipeline

**COMP647 Assignment 02 - Student ID: 1163127**

This notebook implements comprehensive data preprocessing for Lending Club loan data analysis.

## 1. Import Libraries and Setup

In [1]:
# Essential data processing libraries
import pandas as pd
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning utilities
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from scipy import stats

# System and utility libraries
import warnings
import os
from pathlib import Path

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
plt.style.use('default')

## 2. Data Loading Functions

In [2]:
def load_sample_data(sample_size='10000'):
    """
    Load sample datasets for development and analysis.
    
    Parameters:
    sample_size (str): Size of sample to load ('1000', '10000', '50000')
    
    Returns:
    tuple: (accepted_df, rejected_df)
    """
    # Define file paths for data loading
    data_path = '../data/processed/'
    accepted_file = f'accepted_sample_{sample_size}.csv'
    rejected_file = f'rejected_sample_{sample_size}.csv'
    
    print(f"Loading sample datasets (size: {sample_size})...")
    
    try:
        # Load accepted loans dataset
        accepted_df = pd.read_csv(os.path.join(data_path, accepted_file))
        print(f"Accepted loans loaded: {accepted_df.shape[0]:,} rows, {accepted_df.shape[1]} columns")
        
        # Load rejected loans dataset
        rejected_df = pd.read_csv(os.path.join(data_path, rejected_file))
        print(f"Rejected loans loaded: {rejected_df.shape[0]:,} rows, {rejected_df.shape[1]} columns")
        
        print("Data loading completed successfully!")
        
        return accepted_df, rejected_df
        
    except FileNotFoundError as e:
        print(f"Error loading files: {e}")
        print("Please ensure data files are in the correct directory")
        return None, None
    except Exception as e:
        print(f"Unexpected error during data loading: {e}")
        return None, None

## 3. Missing Value Analysis

In [3]:
def analyze_missing_values(df):
    """
    Comprehensive analysis of missing values in the dataset.
    
    Parameters:
    df (pd.DataFrame): Input dataframe
    
    Returns:
    dict: Missing value analysis results
    """
    print(f"Analyzing missing values for dataset with shape: {df.shape}")
    
    # Calculate missing values for each column
    missing_data = df.isnull().sum()
    missing_percentage = (missing_data / len(df)) * 100
    
    # Create detailed missing data summary
    missing_summary = pd.DataFrame({
        'Column': missing_data.index,
        'Missing_Count': missing_data.values,
        'Missing_Percentage': missing_percentage.values,
        'Data_Type': df.dtypes.values,
        'Unique_Values': [df[col].nunique() for col in df.columns]
    })
    
    # Filter only columns with missing values
    missing_summary = missing_summary[missing_summary['Missing_Count'] > 0]
    missing_summary = missing_summary.sort_values('Missing_Percentage', ascending=False)
    
    # Calculate summary statistics
    total_missing = missing_data.sum()
    columns_with_missing = len(missing_summary)
    overall_missing_pct = (total_missing / (len(df) * len(df.columns))) * 100
    
    print(f"Columns with missing values: {columns_with_missing}")
    print(f"Total missing values: {total_missing:,}")
    print(f"Overall missing percentage: {overall_missing_pct:.2f}%")
    
    # Categorize missing values by severity
    analysis_results = {
        'missing_summary': missing_summary,
        'total_missing_values': total_missing,
        'columns_with_missing': columns_with_missing,
        'overall_missing_percentage': overall_missing_pct,
        'structural_missing_90plus': len(missing_summary[missing_summary['Missing_Percentage'] > 90]),
        'high_missing_50_90': len(missing_summary[(missing_summary['Missing_Percentage'] > 50) & (missing_summary['Missing_Percentage'] <= 90)]),
        'moderate_missing_10_50': len(missing_summary[(missing_summary['Missing_Percentage'] > 10) & (missing_summary['Missing_Percentage'] <= 50)]),
        'low_missing_under_10': len(missing_summary[missing_summary['Missing_Percentage'] <= 10])
    }
    
    if columns_with_missing > 0:
        print("\nMissing value categorization:")
        print(f"  Structural missing (>90%): {analysis_results['structural_missing_90plus']} columns")
        print(f"  High missing (50-90%): {analysis_results['high_missing_50_90']} columns") 
        print(f"  Moderate missing (10-50%): {analysis_results['moderate_missing_10_50']} columns")
        print(f"  Low missing (<10%): {analysis_results['low_missing_under_10']} columns")
        
        print(f"\nTop 10 columns with highest missing percentages:")
        if len(missing_summary) > 0:
            for _, row in missing_summary.head(10).iterrows():
                print(f"  {row['Column']}: {row['Missing_Percentage']:.1f}% ({row['Missing_Count']:,} values)")
    else:
        print("No missing values found in dataset")
    
    return analysis_results

## 4. Data Preprocessing Pipeline

In [4]:
def preprocess_lending_data(df):
    """
    Main preprocessing pipeline for Lending Club data.
    
    Parameters:
    df (pd.DataFrame): Raw lending data
    
    Returns:
    pd.DataFrame: Preprocessed data
    """
    print(f"Starting preprocessing pipeline for dataset: {df.shape}")
    
    # Step 1: Create a copy to avoid modifying original data
    df_processed = df.copy()
    
    # Step 2: Basic data validation
    print("Step 1: Basic data validation")
    print(f"  Original shape: {df_processed.shape}")
    print(f"  Memory usage: {df_processed.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Step 3: Remove duplicate rows
    print("Step 2: Duplicate removal")
    initial_rows = len(df_processed)
    df_processed = df_processed.drop_duplicates()
    duplicates_removed = initial_rows - len(df_processed)
    print(f"  Duplicates removed: {duplicates_removed:,}")
    
    # Step 4: Analyze missing values
    print("Step 3: Missing value analysis")
    missing_analysis = analyze_missing_values(df_processed)
    
    # Step 5: Handle missing values based on analysis
    print("Step 4: Missing value treatment")
    df_processed = handle_missing_values(df_processed, missing_analysis)
    
    # Step 6: Handle outliers if requested
    print("Step 5: Outlier handling")
    numeric_columns = df_processed.select_dtypes(include=[np.number]).columns[:5]  # Process first 5 numeric columns for demo
    if len(numeric_columns) > 0:
        df_processed = handle_outliers(df_processed, columns=numeric_columns, method='iqr', action='cap')
    else:
        print("  No numeric columns found for outlier handling")
    
    # Step 7: Data type optimization preparation
    print("Step 6: Data type optimization")
    print(f"  Current data types: {df_processed.dtypes.value_counts().to_dict()}")
    
    print(f"Preprocessing pipeline completed. Final shape: {df_processed.shape}")
    
    return df_processed

## 5. Outlier Detection Functions

Outlier detection using statistical methods for data quality assessment.

In [5]:
def detect_outliers_iqr(df, columns=None, multiplier=1.5):
    """
    Detect outliers using Interquartile Range (IQR) method
    
    This function identifies outliers by calculating the IQR for numeric columns
    and flagging values that fall outside the bounds defined by Q1 - multiplier*IQR
    and Q3 + multiplier*IQR.
    
    Parameters:
    df (DataFrame): Input dataset for outlier analysis
    columns (list): List of column names to analyze (None for all numeric columns)  
    multiplier (float): IQR multiplier for outlier threshold (default: 1.5)
    
    Returns:
    dict: Dictionary containing outlier information for each analyzed column
    """
    if columns is None:
        columns = df.select_dtypes(include=[np.number]).columns
    
    outlier_info = {}
    
    for col in columns:
        if col in df.columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            
            lower_bound = Q1 - multiplier * IQR
            upper_bound = Q3 + multiplier * IQR
            
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
            
            outlier_info[col] = {
                'count': len(outliers),
                'percentage': (len(outliers) / len(df)) * 100,
                'lower_bound': lower_bound,
                'upper_bound': upper_bound,
                'Q1': Q1,
                'Q3': Q3,
                'IQR': IQR
            }
    
    return outlier_info

def detect_outliers_zscore(df, columns=None, threshold=3):
    """
    Detect outliers using Z-score method
    
    This function identifies outliers using standardized Z-scores, flagging values
    that have an absolute Z-score greater than the specified threshold.
    
    Parameters:
    df (DataFrame): Input dataset for outlier analysis
    columns (list): List of column names to analyze (None for all numeric columns)
    threshold (float): Z-score threshold for outlier detection (default: 3)
    
    Returns:
    dict: Dictionary containing outlier information for each analyzed column
    """
    if columns is None:
        columns = df.select_dtypes(include=[np.number]).columns
    
    outlier_info = {}
    
    for col in columns:
        if col in df.columns:
            z_scores = np.abs(stats.zscore(df[col].dropna()))
            outliers_mask = z_scores > threshold
            outlier_count = outliers_mask.sum()
            
            outlier_info[col] = {
                'count': outlier_count,
                'percentage': (outlier_count / len(df[col].dropna())) * 100,
                'threshold': threshold,
                'max_zscore': z_scores.max() if len(z_scores) > 0 else 0
            }
    
    return outlier_info

print("Outlier detection functions defined successfully")
print("Available methods: detect_outliers_iqr(), detect_outliers_zscore()")

Outlier detection functions defined successfully
Available methods: detect_outliers_iqr(), detect_outliers_zscore()


## 6. Missing Value and Outlier Treatment

Implementation of various missing value imputation and outlier handling methods.

In [6]:
def handle_missing_values(df, missing_analysis, strategy='auto'):
    """
    Handle missing values using appropriate imputation strategies
    
    This function applies different imputation methods based on missing value
    percentage and data types. For columns with >90% missing data, it removes
    them. For others, it applies median for numeric and mode for categorical.
    
    Parameters:
    df (DataFrame): Dataset to process
    missing_analysis (dict): Results from analyze_missing_values function
    strategy (str): Imputation strategy ('auto', 'mean', 'median', 'mode')
    
    Returns:
    DataFrame: Dataset with missing values handled
    """
    print(f"Starting missing value treatment with strategy: {strategy}")
    
    df_treated = df.copy()
    columns_dropped = []
    columns_imputed = {}
    
    if missing_analysis['columns_with_missing'] == 0:
        print("  No missing values to handle")
        return df_treated
    
    missing_summary = missing_analysis['missing_summary']
    
    for _, row in missing_summary.iterrows():
        col_name = row['Column']
        missing_pct = row['Missing_Percentage']
        data_type = str(row['Data_Type'])
        
        # Drop columns with >90% missing values
        if missing_pct > 90:
            if col_name in df_treated.columns:
                df_treated = df_treated.drop(columns=[col_name])
                columns_dropped.append(col_name)
                print(f"  Dropped {col_name}: {missing_pct:.1f}% missing")
            continue
        
        # Handle numeric columns
        if 'int' in data_type or 'float' in data_type:
            if strategy == 'auto' or strategy == 'median':
                fill_value = df_treated[col_name].median()
                imputation_method = 'median'
            elif strategy == 'mean':
                fill_value = df_treated[col_name].mean()
                imputation_method = 'mean'
            
            df_treated[col_name] = df_treated[col_name].fillna(fill_value)
            columns_imputed[col_name] = imputation_method
            print(f"  Imputed {col_name}: {imputation_method} = {fill_value:.2f}")
        
        # Handle categorical columns
        else:
            mode_value = df_treated[col_name].mode()
            if len(mode_value) > 0:
                fill_value = mode_value[0]
                df_treated[col_name] = df_treated[col_name].fillna(fill_value)
                columns_imputed[col_name] = 'mode'
                print(f"  Imputed {col_name}: mode = {fill_value}")
            else:
                # Handle case where mode cannot be determined
                df_treated[col_name] = df_treated[col_name].fillna('Unknown')
                columns_imputed[col_name] = 'unknown'
                print(f"  Imputed {col_name}: fallback = Unknown")
    
    print(f"Missing value treatment completed:")
    print(f"  Columns dropped: {len(columns_dropped)}")
    print(f"  Columns imputed: {len(columns_imputed)}")
    print(f"  Final shape: {df_treated.shape}")
    
    return df_treated

def handle_outliers(df, columns=None, method='iqr', action='cap'):
    """
    Handle outliers in numeric columns using various methods
    
    Parameters:
    df (DataFrame): Dataset to process
    columns (list): Columns to process (None for all numeric)
    method (str): Detection method ('iqr', 'zscore')
    action (str): Action to take ('cap', 'remove')
    
    Returns:
    DataFrame: Dataset with outliers handled
    """
    print(f"Starting outlier handling: method={method}, action={action}")
    
    df_treated = df.copy()
    
    if columns is None:
        columns = df_treated.select_dtypes(include=[np.number]).columns
    
    outliers_handled = 0
    
    for col in columns:
        if col in df_treated.columns:
            if method == 'iqr':
                Q1 = df_treated[col].quantile(0.25)
                Q3 = df_treated[col].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                
                outlier_mask = (df_treated[col] < lower_bound) | (df_treated[col] > upper_bound)
                
            elif method == 'zscore':
                z_scores = np.abs(stats.zscore(df_treated[col].dropna()))
                outlier_mask = pd.Series([False] * len(df_treated))
                valid_indices = df_treated[col].dropna().index
                outlier_mask.loc[valid_indices] = z_scores > 3
            
            outlier_count = outlier_mask.sum()
            
            if outlier_count > 0:
                if action == 'cap' and method == 'iqr':
                    # Cap outliers to bounds
                    df_treated.loc[df_treated[col] < lower_bound, col] = lower_bound
                    df_treated.loc[df_treated[col] > upper_bound, col] = upper_bound
                    print(f"  Capped {outlier_count} outliers in {col}")
                
                elif action == 'remove':
                    # Remove outlier rows
                    df_treated = df_treated[~outlier_mask]
                    print(f"  Removed {outlier_count} outlier rows for {col}")
                
                outliers_handled += outlier_count
    
    print(f"Outlier handling completed:")
    print(f"  Total outliers processed: {outliers_handled}")
    print(f"  Final shape: {df_treated.shape}")
    
    return df_treated

print("Missing value and outlier handling functions defined successfully")

Missing value and outlier handling functions defined successfully


## 7. Main Execution

This section demonstrates the complete preprocessing workflow using the implemented functions.

In [7]:
# Load sample data for demonstration
print("=== LOADING SAMPLE DATA ===")
df_accepted, df_rejected = load_sample_data(sample_size='10000')

if df_accepted is not None and df_rejected is not None:
    print(f"\nAccepted loans dataset shape: {df_accepted.shape}")
    print(f"Rejected loans dataset shape: {df_rejected.shape}")
    
    # Display basic info about accepted dataset
    print("\n=== ACCEPTED DATASET OVERVIEW ===")
    print(f"Data types: {df_accepted.dtypes.value_counts().to_dict()}")
    print(f"Memory usage: {df_accepted.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    print("\nFirst few columns:")
    print(df_accepted.columns[:10].tolist())
else:
    print("Failed to load data - check file paths and ensure sample files exist")

=== LOADING SAMPLE DATA ===
Loading sample datasets (size: 10000)...
Accepted loans loaded: 10,000 rows, 151 columns


Rejected loans loaded: 10,000 rows, 9 columns
Data loading completed successfully!

Accepted loans dataset shape: (10000, 151)
Rejected loans dataset shape: (10000, 9)

=== ACCEPTED DATASET OVERVIEW ===
Data types: {dtype('float64'): 114, dtype('O'): 36, dtype('int64'): 1}
Memory usage: 27.37 MB

First few columns:
['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade']


In [8]:
# Analyze missing values in the accepted dataset
if df_accepted is not None:
    print("=== MISSING VALUE ANALYSIS ===")
    missing_results = analyze_missing_values(df_accepted)
    
    # Display detailed results
    print(f"\nAnalysis Summary:")
    print(f"- Total columns with missing data: {missing_results['columns_with_missing']}")
    print(f"- Overall missing percentage: {missing_results['overall_missing_percentage']:.2f}%")
    
    if missing_results['columns_with_missing'] > 0:
        print(f"\nMissing data severity breakdown:")
        print(f"- Structural missing (>90%): {missing_results['structural_missing_90plus']} columns")
        print(f"- High missing (50-90%): {missing_results['high_missing_50_90']} columns")
        print(f"- Moderate missing (10-50%): {missing_results['moderate_missing_10_50']} columns") 
        print(f"- Low missing (<10%): {missing_results['low_missing_under_10']} columns")
        
        # Show the missing summary dataframe if it exists
        if len(missing_results['missing_summary']) > 0:
            print(f"\nTop 5 columns with highest missing percentages:")
            top_missing = missing_results['missing_summary'].head(5)
            for _, row in top_missing.iterrows():
                print(f"- {row['Column']}: {row['Missing_Percentage']:.1f}% missing ({row['Data_Type']})")
else:
    print("No data available for missing value analysis")

=== MISSING VALUE ANALYSIS ===
Analyzing missing values for dataset with shape: (10000, 151)
Columns with missing values: 70
Total missing values: 424,863
Overall missing percentage: 28.14%

Missing value categorization:
  Structural missing (>90%): 38 columns
  High missing (50-90%): 5 columns
  Moderate missing (10-50%): 3 columns
  Low missing (<10%): 24 columns

Top 10 columns with highest missing percentages:
  member_id: 100.0% (10,000 values)
  revol_bal_joint: 100.0% (10,000 values)
  sec_app_fico_range_high: 100.0% (10,000 values)
  sec_app_earliest_cr_line: 100.0% (10,000 values)
  sec_app_inq_last_6mths: 100.0% (10,000 values)
  sec_app_mort_acc: 100.0% (10,000 values)
  sec_app_open_acc: 100.0% (10,000 values)
  sec_app_revol_util: 100.0% (10,000 values)
  sec_app_open_act_il: 100.0% (10,000 values)
  sec_app_num_rev_accts: 100.0% (10,000 values)

Analysis Summary:
- Total columns with missing data: 70
- Overall missing percentage: 28.14%

Missing data severity breakdown:
-

In [9]:
# Run the complete preprocessing pipeline
if df_accepted is not None:
    print("=== PREPROCESSING PIPELINE DEMONSTRATION ===")
    df_processed = preprocess_lending_data(df_accepted)
    
    print(f"\n=== PREPROCESSING RESULTS ===")
    print(f"Original shape: {df_accepted.shape}")
    print(f"Processed shape: {df_processed.shape}")
    
    # Compare memory usage
    original_memory = df_accepted.memory_usage(deep=True).sum() / 1024**2
    processed_memory = df_processed.memory_usage(deep=True).sum() / 1024**2
    print(f"Original memory usage: {original_memory:.2f} MB")
    print(f"Processed memory usage: {processed_memory:.2f} MB")
    
    if df_processed.shape[0] < df_accepted.shape[0]:
        rows_removed = df_accepted.shape[0] - df_processed.shape[0]
        print(f"Rows removed during processing: {rows_removed:,}")
    
    print("\nPreprocessing pipeline demonstration completed!")
else:
    print("No data available for preprocessing pipeline demonstration")

=== PREPROCESSING PIPELINE DEMONSTRATION ===
Starting preprocessing pipeline for dataset: (10000, 151)
Step 1: Basic data validation
  Original shape: (10000, 151)
  Memory usage: 27.37 MB
Step 2: Duplicate removal
  Duplicates removed: 0
Step 3: Missing value analysis
Analyzing missing values for dataset with shape: (10000, 151)


Columns with missing values: 70
Total missing values: 424,863
Overall missing percentage: 28.14%

Missing value categorization:
  Structural missing (>90%): 38 columns
  High missing (50-90%): 5 columns
  Moderate missing (10-50%): 3 columns
  Low missing (<10%): 24 columns

Top 10 columns with highest missing percentages:
  member_id: 100.0% (10,000 values)
  revol_bal_joint: 100.0% (10,000 values)
  sec_app_fico_range_high: 100.0% (10,000 values)
  sec_app_earliest_cr_line: 100.0% (10,000 values)
  sec_app_inq_last_6mths: 100.0% (10,000 values)
  sec_app_mort_acc: 100.0% (10,000 values)
  sec_app_open_acc: 100.0% (10,000 values)
  sec_app_revol_util: 100.0% (10,000 values)
  sec_app_open_act_il: 100.0% (10,000 values)
  sec_app_num_rev_accts: 100.0% (10,000 values)
Step 4: Missing value treatment
Starting missing value treatment with strategy: auto
  Dropped member_id: 100.0% missing


  Dropped revol_bal_joint: 100.0% missing
  Dropped sec_app_fico_range_high: 100.0% missing
  Dropped sec_app_earliest_cr_line: 100.0% missing
  Dropped sec_app_inq_last_6mths: 100.0% missing
  Dropped sec_app_mort_acc: 100.0% missing
  Dropped sec_app_open_acc: 100.0% missing
  Dropped sec_app_revol_util: 100.0% missing
  Dropped sec_app_open_act_il: 100.0% missing
  Dropped sec_app_num_rev_accts: 100.0% missing


  Dropped sec_app_chargeoff_within_12_mths: 100.0% missing
  Dropped sec_app_collections_12_mths_ex_med: 100.0% missing
  Dropped sec_app_mths_since_last_major_derog: 100.0% missing
  Dropped sec_app_fico_range_low: 100.0% missing
  Dropped desc: 100.0% missing
  Dropped dti_joint: 99.4% missing
  Dropped verification_status_joint: 99.4% missing
  Dropped annual_inc_joint: 99.4% missing
  Dropped orig_projected_additional_accrued_interest: 99.3% missing
  Dropped hardship_type: 99.2% missing
  Dropped hardship_last_payment_amount: 99.2% missing
  Dropped hardship_payoff_balance_amount: 99.2% missing
  Dropped hardship_dpd: 99.2% missing
  Dropped hardship_loan_status: 99.2% missing
  Dropped hardship_reason: 99.2% missing
  Dropped hardship_status: 99.2% missing
  Dropped deferral_term: 99.2% missing
  Dropped hardship_amount: 99.2% missing
  Dropped hardship_start_date: 99.2% missing
  Dropped payment_plan_start_date: 99.2% missing
  Dropped hardship_length: 99.2% missing
  Dropped ha

  Imputed mths_since_last_record: median = 67.00
  Imputed mths_since_recent_bc_dlq: median = 36.00
  Imputed mths_since_last_major_derog: median = 44.00
  Imputed mths_since_recent_revol_delinq: median = 32.00
  Imputed mths_since_last_delinq: median = 31.00
  Imputed il_util: median = 75.00
  Imputed mths_since_recent_inq: median = 5.00
  Imputed num_tl_120dpd_2m: median = 0.00
  Imputed emp_title: mode = Teacher
  Imputed emp_length: mode = 10+ years
  Imputed mths_since_rcnt_il: median = 12.00
  Imputed mo_sin_old_il_acct: median = 130.00
  Imputed title: mode = Debt consolidation
  Imputed bc_util: median = 61.20
  Imputed percent_bc_gt_75: median = 40.00
  Imputed bc_open_to_buy: median = 5222.00
  Imputed mths_since_recent_bc: median = 13.00
  Imputed last_pymnt_d: mode = Jan-2019
  Imputed revol_util: median = 50.80


  Imputed open_acc_6m: median = 1.00
  Imputed inq_last_12m: median = 2.00
  Imputed open_act_il: median = 2.00
  Imputed total_cu_tl: median = 0.00
  Imputed open_il_24m: median = 1.00
  Imputed total_bal_il: median = 25198.00
  Imputed open_rv_12m: median = 1.00
  Imputed open_rv_24m: median = 2.00
  Imputed max_bal_bc: median = 4533.00
  Imputed all_util: median = 61.00
  Imputed inq_fi: median = 0.00
  Imputed open_il_12m: median = 1.00
Missing value treatment completed:
  Columns dropped: 38
  Columns imputed: 32
  Final shape: (10000, 113)
Step 5: Outlier handling
Starting outlier handling: method=iqr, action=cap
  Capped 419 outliers in id
  Capped 120 outliers in int_rate
Outlier handling completed:
  Total outliers processed: 539
  Final shape: (10000, 113)
Step 6: Data type optimization
  Current data types: {dtype('float64'): 88, dtype('O'): 24, dtype('int64'): 1}
Preprocessing pipeline completed. Final shape: (10000, 113)

=== PREPROCESSING RESULTS ===
Original shape: (1000

Original memory usage: 27.37 MB
Processed memory usage: 22.00 MB

Preprocessing pipeline demonstration completed!


In [10]:
# Comprehensive preprocessing workflow demonstration
if df_accepted is not None:
    print("=== COMPREHENSIVE PREPROCESSING WORKFLOW ===")
    
    # Step 1: Initial data assessment
    print(f"\nStep 1: Initial Assessment")
    print(f"Dataset shape: {df_accepted.shape}")
    print(f"Memory usage: {df_accepted.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Step 2: Run complete preprocessing pipeline
    print(f"\nStep 2: Complete Preprocessing Pipeline")
    df_fully_processed = preprocess_lending_data(df_accepted)
    
    # Step 3: Compare before and after
    print(f"\n=== PREPROCESSING RESULTS SUMMARY ===")
    print(f"Original dataset:")
    print(f"  - Shape: {df_accepted.shape}")
    print(f"  - Missing values: {df_accepted.isnull().sum().sum():,}")
    print(f"  - Memory: {df_accepted.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    print(f"\nProcessed dataset:")
    print(f"  - Shape: {df_fully_processed.shape}")
    print(f"  - Missing values: {df_fully_processed.isnull().sum().sum():,}")
    print(f"  - Memory: {df_fully_processed.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Step 4: Calculate improvement metrics
    rows_change = df_fully_processed.shape[0] - df_accepted.shape[0]
    cols_change = df_fully_processed.shape[1] - df_accepted.shape[1]
    missing_reduction = df_accepted.isnull().sum().sum() - df_fully_processed.isnull().sum().sum()
    
    print(f"\nPreprocessing Impact:")
    print(f"  - Rows changed: {rows_change:,} ({rows_change/df_accepted.shape[0]*100:.1f}%)")
    print(f"  - Columns changed: {cols_change:,}")
    print(f"  - Missing values eliminated: {missing_reduction:,}")
    
    # Step 5: Data quality assessment
    print(f"\nData Quality Assessment:")
    completeness = (1 - df_fully_processed.isnull().sum().sum() / (df_fully_processed.shape[0] * df_fully_processed.shape[1])) * 100
    print(f"  - Data completeness: {completeness:.2f}%")
    print(f"  - Ready for analysis: {'Yes' if completeness > 95 else 'Needs review'}")
    
    print("\n=== PREPROCESSING WORKFLOW COMPLETED SUCCESSFULLY ===")
    print("Dataset is now ready for exploratory data analysis and modeling")
else:
    print("No data available for comprehensive preprocessing demonstration")

=== COMPREHENSIVE PREPROCESSING WORKFLOW ===

Step 1: Initial Assessment
Dataset shape: (10000, 151)
Memory usage: 27.37 MB

Step 2: Complete Preprocessing Pipeline
Starting preprocessing pipeline for dataset: (10000, 151)
Step 1: Basic data validation
  Original shape: (10000, 151)


  Memory usage: 27.37 MB


Step 2: Duplicate removal
  Duplicates removed: 0
Step 3: Missing value analysis
Analyzing missing values for dataset with shape: (10000, 151)
Columns with missing values: 70
Total missing values: 424,863
Overall missing percentage: 28.14%

Missing value categorization:
  Structural missing (>90%): 38 columns
  High missing (50-90%): 5 columns
  Moderate missing (10-50%): 3 columns
  Low missing (<10%): 24 columns

Top 10 columns with highest missing percentages:
  member_id: 100.0% (10,000 values)
  revol_bal_joint: 100.0% (10,000 values)
  sec_app_fico_range_high: 100.0% (10,000 values)
  sec_app_earliest_cr_line: 100.0% (10,000 values)
  sec_app_inq_last_6mths: 100.0% (10,000 values)
  sec_app_mort_acc: 100.0% (10,000 values)
  sec_app_open_acc: 100.0% (10,000 values)
  sec_app_revol_util: 100.0% (10,000 values)
  sec_app_open_act_il: 100.0% (10,000 values)
  sec_app_num_rev_accts: 100.0% (10,000 values)
Step 4: Missing value treatment
Starting missing value treatment with strategy

  Dropped member_id: 100.0% missing
  Dropped revol_bal_joint: 100.0% missing
  Dropped sec_app_fico_range_high: 100.0% missing
  Dropped sec_app_earliest_cr_line: 100.0% missing
  Dropped sec_app_inq_last_6mths: 100.0% missing
  Dropped sec_app_mort_acc: 100.0% missing
  Dropped sec_app_open_acc: 100.0% missing
  Dropped sec_app_revol_util: 100.0% missing
  Dropped sec_app_open_act_il: 100.0% missing
  Dropped sec_app_num_rev_accts: 100.0% missing
  Dropped sec_app_chargeoff_within_12_mths: 100.0% missing
  Dropped sec_app_collections_12_mths_ex_med: 100.0% missing


  Dropped sec_app_mths_since_last_major_derog: 100.0% missing
  Dropped sec_app_fico_range_low: 100.0% missing
  Dropped desc: 100.0% missing
  Dropped dti_joint: 99.4% missing
  Dropped verification_status_joint: 99.4% missing
  Dropped annual_inc_joint: 99.4% missing
  Dropped orig_projected_additional_accrued_interest: 99.3% missing
  Dropped hardship_type: 99.2% missing
  Dropped hardship_last_payment_amount: 99.2% missing
  Dropped hardship_payoff_balance_amount: 99.2% missing
  Dropped hardship_dpd: 99.2% missing


  Dropped hardship_loan_status: 99.2% missing
  Dropped hardship_reason: 99.2% missing
  Dropped hardship_status: 99.2% missing
  Dropped deferral_term: 99.2% missing
  Dropped hardship_amount: 99.2% missing
  Dropped hardship_start_date: 99.2% missing
  Dropped payment_plan_start_date: 99.2% missing
  Dropped hardship_length: 99.2% missing
  Dropped hardship_end_date: 99.2% missing
  Dropped debt_settlement_flag_date: 97.2% missing
  Dropped settlement_status: 97.2% missing
  Dropped settlement_date: 97.2% missing
  Dropped settlement_amount: 97.2% missing
  Dropped settlement_percentage: 97.2% missing
  Dropped settlement_term: 97.2% missing
  Imputed next_pymnt_d: mode = Apr-2019
  Imputed mths_since_last_record: median = 67.00
  Imputed mths_since_recent_bc_dlq: median = 36.00
  Imputed mths_since_last_major_derog: median = 44.00
  Imputed mths_since_recent_revol_delinq: median = 32.00
  Imputed mths_since_last_delinq: median = 31.00
  Imputed il_util: median = 75.00
  Imputed mths

  Capped 419 outliers in id
  Capped 120 outliers in int_rate
Outlier handling completed:
  Total outliers processed: 539
  Final shape: (10000, 113)
Step 6: Data type optimization
  Current data types: {dtype('float64'): 88, dtype('O'): 24, dtype('int64'): 1}
Preprocessing pipeline completed. Final shape: (10000, 113)

=== PREPROCESSING RESULTS SUMMARY ===
Original dataset:
  - Shape: (10000, 151)
  - Missing values: 424,863


  - Memory: 27.37 MB

Processed dataset:
  - Shape: (10000, 113)
  - Missing values: 0


  - Memory: 22.00 MB

Preprocessing Impact:
  - Rows changed: 0 (0.0%)
  - Columns changed: -38
  - Missing values eliminated: 424,863

Data Quality Assessment:
  - Data completeness: 100.00%
  - Ready for analysis: Yes

=== PREPROCESSING WORKFLOW COMPLETED SUCCESSFULLY ===
Dataset is now ready for exploratory data analysis and modeling
